![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb)

# 24. Improved Entity Resolvers in Spark NLP with sBert

In [ ]:
!python3 -m pip install --upgrade spark-nlp-jsl==2.7.2 --user --extra-index-url https://pypi.johnsnowlabs.com/$secret

In [135]:
import sparknlp

sparknlp.version()

'2.6.5'

In [136]:
import sparknlp_jsl

sparknlp_jsl.version()

'2.7.2'

In [1]:
keyfile = 'workshop_license_keys_365.json'

import os
import json
import pandas as pd
import numpy as np
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

with open(keyfile, 'r') as f:
    license_keys = json.load(f)


In [2]:
secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
jsl_version = license_keys['JSL_VERSION']
version = license_keys['PUBLIC_VERSION']

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import sparknlp_jsl
import sys, os, time
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel


params = {"spark.driver.memory":"32G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}
 
spark = sparknlp_jsl.start(secret, params=params)


In [3]:
spark

## Snomed pipeline

In [163]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("snomed_code")\
  .setDistanceFunction("EUCLIDEAN")

snomed_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver])

snomed_lp = LightPipeline(snomed_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings download started this may take some time.
Approximate size to download 1.4 GB
[OK!]


## ICD10CM pipeline

In [16]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("icd10cm_code")\
  .setDistanceFunction("EUCLIDEAN")

icd_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver])

icd_lp = LightPipeline(icd_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## RxNorm pipeline

In [23]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("rxnorm_code")\
  .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm download started this may take some time.
Approximate size to download 810.7 MB
[OK!]


## CPT pipeline

In [153]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("cpt_code")\
  .setDistanceFunction("EUCLIDEAN")

cpt_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])

cpt_lp = LightPipeline(cpt_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_cpt_procedures_augmented download started this may take some time.
Approximate size to download 112.2 MB
[OK!]


## all the resolvers in the same pipeline 
### (just to show how it is done..  will not be used in this notebook)

In [ ]:

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("snomed_code")\
  .setDistanceFunction("EUCLIDEAN")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("icd10cm_code")\
  .setDistanceFunction("EUCLIDEAN")

rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("rxnorm_code")\
  .setDistanceFunction("EUCLIDEAN")

cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("cpt_code")\
  .setDistanceFunction("EUCLIDEAN")

resolver_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver,
        icd10_resolver,
        rxnorm_resolver,
        cpt_resolver])

resolver_lp = LightPipeline(resolver_pipelineModel)


## utility functions

In [154]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code'):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    entities=[]
    all_distances =[]
    all_codes=[]
    
    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        entities.append(chunk.metadata['entity']) 
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'entity':entities,
                    'code':codes,'all_codes':all_codes, 
                    'resolutions':resolutions, 'all_distances':all_distances})

    
    return df



## getting some predictions from resolvers

In [50]:
text = 'bladder cancer'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 9.88 ms, sys: 5.1 ms, total: 15 ms
Wall time: 704 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 7.4914, 7.9359, 8.2425, 9.0119]"


In [138]:
text = 'severe stomach pain'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 8.01 ms, sys: 6.76 ms, total: 14.8 ms
Wall time: 708 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,severe stomach pain,0,18,None,R101,"[R101, R109, R1082, M791, R104, R1084, K388, R529, F454, R079, R53, R1013, R52, M7918, R51, R520, R4589, R1033, R072, R103, R1010]","[upper abdominal pain, intractable abdominal pain, epigastric pain, sore pain, generalized colicky abdominal pain, generalized abdominal pain, colicky abdominal pain, severe pain, psychosomatic abdominal pain, upper chest pain, exhausting with pain, burning epigastric pain, heavy pain, abdominal muscle pain, head pain, horrible present pain, cruel with pain, central abdominal pain, esophageal pain, o/e - epigastric pain, ulcer-type pain]","[6.0181, 6.1794, 6.2316, 6.3038, 6.3213, 6.3808, 6.4652, 6.5255, 6.6239, 6.6474, 6.6767, 6.7574, 6.7577, 6.7744, 6.8206, 6.8932, 6.9044, 7.0089, 7.0235, 7.0339, 7.0490]"


In [51]:
text = 'bladder cancer'

%time get_codes (snomed_lp, text, vocab='snomed_code')

CPU times: user 6.3 ms, sys: 8.86 ms, total: 15.2 ms
Wall time: 773 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 5.8576, 6.4952, 6.4952, 6.4952, 7.1698, 7.4762, 7.5159, 7.5159, 7.6426, 7.9359, 8.2425]"


In [52]:
text = '67 yrs-old'

%time get_codes (snomed_lp, text, vocab='snomed_code')

CPU times: user 5.77 ms, sys: 9.53 ms, total: 15.3 ms
Wall time: 771 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,67 yrs-old,0,9,None,102527003,"[102527003, 102528008, 2391000124102, 111522004, 28288005, 49808004, 260278007, 248280005, 102891000, 152003002, 183774000, 308508003, 249860006, 246944005]","[age 60 to 64 years (finding), age more than 65 years (finding), universal designation 67, gerontoxon, middle-age, geriatric state, 6/60 (finding), aging, age-related cognitive decline (finding), listed for geriatrics admiss'n, listed for geriatrics admiss'n, geriatric monitoring status (finding), posture appropriate for age, pseudo-gerontoxon]","[11.4971, 11.5051, 11.7625, 12.4103, 12.6651, 12.9846, 13.0190, 13.1351, 13.2770, 13.3563, 13.3563, 13.3586, 13.3714, 13.3961]"


In [53]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 11.7 ms, sys: 3.59 ms, total: 15.3 ms
Wall time: 468 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,None,406081,"[406081, 576612, 403968, 861024, 404727, 334738, 332848, 861026, 333262, 439563, 450523, 1744000, 484793, 402346, 1726496, 316350, 858858, 336846, 316844, 1946837, 451225, 328507, 437723, 385601, 315677]","[metformin 100 mg/ml, metformin 100 mg/ml [riomet], metformin 100 mg/ml oral solution, metformin hydrochloride 100 mg/ml, metformin 100 mg/ml oral solution [riomet], fenofibrate 100 mg, ciprofibrate 100 mg, metformin hydrochloride 100 mg/ml [riomet], rutin 100 mg, fendiline 100 mg, perazine 100 mg, emtricitabine 100 mg, solifenacin 100 mg, miglustat 100 mg, azacitidine 100 mg, niacin 100 mg, carnosine 100 mg, trimebutine 100 mg, torsemide 100 mg, abemaciclib 100 mg, pyrantel 100 mg, rimantadine 100 mg, azintamide 100 mg, mebeverine 100 mg, cimetidine 100 mg]","[3.8387, 4.8433, 5.6433, 6.3835, 6.4686, 6.5293, 6.5728, 6.9061, 6.9297, 6.9512, 7.0943, 7.1014, 7.1202, 7.1323, 7.1327, 7.1425, 7.2076, 7.2118, 7.2518, 7.2531, 7.2533, 7.2761, 7.2802, 7.2915, 7.3286]"


In [137]:
text = 'heart surgery'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 6.68 ms, sys: 9.15 ms, total: 15.8 ms
Wall time: 103 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,heart surgery,0,12,None,33257,"[33257, 33258, 33259, 33999, 0306T, 0304T, 0302T, 0167T, 93583, 1006057, 62319]","[Cardiac surgery procedure, Cardiac surgery procedure, Cardiac surgery procedure, Cardiac surgery procedure, Heart procedure, Heart procedure, Heart procedure, Heart implantation, Operation on cardiac septum, Surgical Procedures on the Heart and Pericardium, Insertion of catheter into heart chamber]","[8.1433, 8.1433, 8.1433, 8.1433, 9.3368, 9.3368, 9.3368, 10.0148, 10.0324, 10.6450, 10.8858]"


In [156]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 12.2 ms, sys: 2.56 ms, total: 14.8 ms
Wall time: 99.8 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,None,62284,"[62284, 74183, 74181, 70481, 70480, 76700, 76705, 0157T, 49329, 49999, 61108, 70540, 70543, 71555, 49080, 49083, 33891, 34051, 34001, 35002]","[CT of spine, MRI of abdomen, MRI of abdomen, CT of orbits, CT of orbits, US abdominal scan, US abdominal scan, Opening of abdomen, Procedure on abdomen, Procedure on abdomen, Centesis, MRI of neck, MRI of neck, MRI of chest, Drainage of abdomen, Drainage of abdomen, Incision of neck, Incision of neck, Incision of neck, Incision of neck]","[8.8537, 9.1304, 9.1304, 9.5856, 9.5856, 9.9397, 9.9397, 9.9516, 10.2509, 10.2509, 10.6676, 10.7085, 10.7085, 10.7766, 10.8118, 10.8118, 10.9787, 10.9787, 10.9787, 10.9787]"


In [157]:
text = 'Left heart cath'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 13.4 ms, sys: 4.48 ms, total: 17.9 ms
Wall time: 105 ms


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,Left heart cath,0,14,None,93462,"[93462, 93607, 0281T, 93620, 93514, 93452, 93565, 93568, 93564, 93567, 93566, 62319]","[Cardiac catheterisation, left heart, Left ventricular recording (Deprecated), LAA - Plication of left atrial appendage, Left ventricular recording, Left heart catheterization by left ventricular puncture, Ventriculography of left ventricle, CCA - Cardiac catheterisation, CCA - Cardiac catheterisation, CCA - Cardiac catheterisation, CCA - Cardiac catheterisation, CCA - Cardiac catheterisation, CCA - Cardiac catheterisation]","[8.6548, 9.8134, 9.8843, 10.0954, 10.3075, 10.5515, 10.6741, 10.6741, 10.6741, 10.6741, 10.6741, 10.6741]"


In [165]:
icd_chunks = ['advanced liver disease',
'advanced lung disease',
'basal cell carcinoma of skin',
'acute maxillary sinusitis',
'chronic kidney disease stage',
'diabetes mellitus type 2',
'lymph nodes of multiple sites',
'other chronic pain',
'severe abdominal pain',
'squamous cell carcinoma of skin',
'type 2 diabetes mellitus']

snomed_chunks= ['anemia', 'adenocarcinoma', 'aortic valve stenosis',
       'atherosclerosis', 'atrial fibrillation', 'dyspnea',
       'hypertension', 'lung cancer', 'seizure',
       'squamous cell carcinoma', 'stage IIIB', 'mediastinal lymph nodes']

In [61]:
from IPython.display import display

for chunk in icd_chunks:

    print ('>> ',chunk)
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))

>>  advanced liver disease


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,advanced liver disease,0,21,None,K7460,"[K7460, K769, K721, K746, R945, Z871, K7402, K729, K7290, K7689, D489, D499, R748, K719]","[advanced cirrhosis, chronic liver disease, chronic hepatic failure, cirrhosis and chronic liver disease, abnormal liver function, h/o: liver disease, hepatic fibrosis, advanced fibrosis, liver function failure, liver failure, abnormal hepatic function, tumor of advanced extent, tumour of advanced extent, elevated liver enzymes level, toxic liver disease]","[6.8937, 8.3979, 8.5299, 9.1728, 9.3652, 9.3766, 9.4469, 9.4751, 9.5000, 9.8048, 9.8914, 9.8951, 10.0151, 10.0640]"


>>  advanced lung disease


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,advanced lung disease,0,20,None,D499,"[D499, D489, J449, H359, J849, J628, K7460, J399, I5083, J961, J989, I509, J8410, R54, J984, I279]","[tumour of advanced extent, tumor of advanced extent, severe chronic obstructive pulmonary disease, advanced retinal disease, chronic infiltrative lung disease, chronic lung disease, advanced cirrhosis, upper respiratory disease, high output heart failure, chronic respiratory failure, chronic respiratory system disease, cardiac failure, progressive lung fibrosis, advanced age, chronic lung disease (disorder), chronic pulmonary heart disease]","[9.2256, 9.4250, 9.5090, 9.5274, 9.5762, 9.5848, 9.9092, 9.9598, 9.9863, 10.1878, 10.2669, 10.2964, 10.3895, 10.4332, 10.4493, 10.4542]"


>>  basal cell carcinoma of skin


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,severe abdominal pain,0,20,None,R1084,"[R1084, R101, R109, K388, R104, R1033, R102, F454, K805, R529, R52, M2551]","[generalized abdominal pain, upper abdominal pain, intractable abdominal pain, colicky abdominal pain, generalized colicky abdominal pain, central abdominal pain, acute abdominal pain, psychosomatic abdominal pain, recurrent abdominal pain, severe pain, heavy pain, shoulder pain from abdomen]","[3.6894, 3.8009, 3.8049, 4.6934, 4.7209, 4.9342, 5.0550, 5.0671, 5.3116, 5.4290, 5.8415, 5.9116]"


>>  squamous cell carcinoma of skin


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma of skin,0,30,None,C449,"[C449, C443, D049, C4492, C4452, C4499, C4432, C809, C4442, C4A70, C444, C4491, C799]","[squamous cell carcinoma of skin, squamous cell carcinoma of skin of face, squamous cell carcinoma in situ of skin, scc - squamous cell carcinoma of skin, squamous cell carcinoma of skin of trunk, mucoepidermoid carcinoma of skin, squamous cell carcinoma of skin of and unsp parts of face, squamous cell carcinoma, squamous cell carcinoma of skin of neck, trabecular cell carcinoma of skin, squamous cell carcinoma of scalp, basal cell carcinoma of skin, squamous cell carcinomatosis]","[0.0000, 3.9417, 4.3020, 4.5867, 4.6218, 5.1354, 5.6903, 5.8818, 5.9843, 6.1362, 6.2912, 6.2998, 6.3445]"


>>  type 2 diabetes mellitus


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus,0,23,None,E119,"[E119, E118, E113, O2411, E139, E117, E1143, E109, E1169, E114, E112, E1151, E1162]","[type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, secondary diabetes mellitus, multiple complications of type 2 diabetes mellitus, neurological disorder with type 2 diabetes mellitus, diabetes mellitus, diabetic dyslipidemia associated with type 2 diabetes mellitus, neurological disorder with diabetes type 2, kidney disorder associated with type 2 diabetes mellitus, peripheral circulatory disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus with skin complications]","[0.0000, 3.7729, 4.8808, 4.9391, 5.1732, 5.2452, 5.5212, 5.5994, 5.7916, 5.9043, 5.9998, 6.0564, 6.0600]"


In [166]:

for chunk in snomed_chunks:

    print ('>> ',chunk)
    display(get_codes (snomed_lp, chunk, vocab='snomed_code'))

>>  anemia


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,anemia,0,5,None,271737000,"[271737000, 64593003, 154786001, 11503009, 191268006, 35372003, 191125001, 267513007, 234347009, 737220002, 234360003, 141336002, 164139008, 84027009, 191139001, 154789008]","[anemia, anemia, anemia, relative anemia, chronic anemia, aids with anemia, deficiency anemias, deficiency anemias, secondary anemia, metabolic anemia, vegan anemia, o/e - anemia, o/e - anemia, pernicious anemia, pernicious anemia, pernicious anemia]","[0.0000, 0.0000, 0.0000, 5.3735, 5.3737, 5.6006, 5.7604, 5.7604, 6.5642, 6.6888, 6.6901, 6.7129, 6.7129, 7.0680, 7.0680, 7.0680]"


>>  adenocarcinoma


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,adenocarcinoma,0,13,None,443961001,"[443961001, 403902008, 423595004, 189578007, 189614008, 423607006, 722688002, 154433003, 269513004, 269546005, 443416007, 403951006, 307599002, 441535001, 307593001, 154576004, 269624009, 188476000, 443261008]","[adenocarcinoma, adenosquamous carcinoma, adenocarcinoma carcinomatosis, [m]adenocarcinomas, adenocarcinoid tumor, adenocarcinoma of anus, carcinoma, carcinoma, carcinoma, anus carcinoma, adenoma, ceruminous gland adenocarcinoma, sebaceous adenocarcinoma, adenocarcinoma of head and neck, carcinomatosis, carcinomatosis, carcinomatosis, carcinomatosis, oxyphilic adenocarcinoma]","[0.0000, 4.9457, 5.7756, 5.9483, 6.0696, 6.8884, 6.9016, 6.9016, 6.9016, 6.9074, 7.1031, 7.3334, 7.4297, 7.4467, 7.5127, 7.5127, 7.5127, 7.5127, 7.6008]"


>>  aortic valve stenosis


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma,0,22,None,402815007,"[402815007, 154605007, 425303004, 254651007, 403904009, 189565007, 403905005, 403902008, 400066006, 403903003, 189568009, 403899005, 254656002, 254657006, 423284006, 403892001, 716659002]","[squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinomatosis, cutaneous squamous cell carcinoma, verrucous squamous cell carcinoma, squamous cell carcinoma in situ, multiple squamous cell carcinomata, adenosquamous cell carcinoma, intraepithelial squamous cell carcinoma, signet ring squamous cell carcinoma, keratinising squamous cell carcinoma, squamous cell carcinoma of skin of trunk, squamous cell carcinoma in situ of skin, squamous cell carcinoma in situ of skin, squamous cell carcinoma of skin of neck, squamous cell carcinoma of skin of face, squamous cell carcinoma of head and neck]","[0.0000, 0.0000, 3.4540, 5.0525, 5.3144, 5.4448, 5.5520, 5.5603, 5.9656, 6.2072, 6.2416, 6.5220, 6.5347, 6.5347, 6.6065, 6.7793, 6.8173]"


>>  stage IIIB


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,stage IIIB,0,9,None,64062008,"[64062008, 73082003, 45597001, 443815000, 50283003, 48105005, 64877001, 443962008, 55336000, 33907000, 433144002, 443817008, 61026006, 52708004, 434002]","[clinical stage 3 b, clinical stage 3 a, group a3b, pt3a,b category, clinical stage iii, clinical stage 3 c, clinical stage iii s, pn3b category, clinical stage iii d, clinical stage iii e, ckd stage 3, pt3b category, g3 stage, figo cc stage iii, ebr 3]","[6.8802, 8.4251, 8.9579, 8.9624, 9.0434, 9.1384, 9.4788, 9.5235, 9.5287, 9.5992, 9.7577, 9.8641, 10.7132, 10.7660, 10.7975]"


>>  mediastinal lymph nodes


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,mediastinal lymph nodes,0,22,None,301296002,"[301296002, 127248001, 49483002, 94147001, 234317007, 52324001, 127245003, 30746006, 274711004, 19616004, 126725000, 301855007, 425061006, 127274007, 76616003, 127232002, 127235000, 127234001]","[mediastinal observation, neoplasm of mediastinal lymph nodes, mediastinal disease, mediastinal mass, lymphangiography of mediastinal lymph nodes, mediastinal lymphadenopathy, neoplasm of intrathoracic lymph nodes, swelling of lymph nodes, mediastinal shift, mediastinal shift, tumour of mediastinum, finding of lymph node, lymphadenopathy of head and/or neck, neoplasm of lymph nodes of multiple sites, disease of lymph node, neoplasm of lymph node, neoplasm of lymph nodes of neck, neoplasm of lymph nodes of face]","[7.3230, 7.3911, 7.8429, 8.5316, 8.9097, 9.0562, 9.1809, 9.1958, 9.2519, 9.2519, 9.4111, 9.4330, 9.4808, 9.5067, 9.5165, 9.6191, 9.6749, 9.6974]"


In [140]:
clinical_chunks = ['bladder cancer',
 'anemia in chronic kidney disease',
 'castleman disease',
 'congestive heart failure',
 'diabetes mellitus type 2',
 'lymph nodes of multiple sites',
 'malignant melanoma of skin',
 'malignant neoplasm of lower lobe, bronchus',
 'metastatic lung cancer',
 'secondary malignant neoplasm of bone',
 'type 2 diabetes mellitus',
 'type 2 diabetes mellitus/insulin',
 'unsp malignant neoplasm of lymph node']


for chunk in clinical_chunks:

    print ('>> ',chunk)
    
    print ('icd10cm_code')
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))
    
    print ('snomed_code')
    display(get_codes (snomed_lp, chunk, vocab='snomed_code'))

>>  bladder cancer
icd10cm_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 7.4914, 7.9359, 8.2425, 9.0119]"


snomed_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,None,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 5.8576, 6.4952, 6.4952, 6.4952, 7.1698, 7.4762, 7.5159, 7.5159, 7.6426, 7.9359, 8.2425]"


>>  anemia in chronic kidney disease
icd10cm_code


...

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus/insulin,0,31,None,237599002,"[237599002, 366909003, 44054006, 368051000119109, 422014003, 422099009, 199230006, 137931000119102, 190384004, 421326000]","[insulin treated type 2 diabetes mellitus, insulin treated type 2 diabetes mellitus, type 2 diabetes mellitus, hyperglycemia due to type 2 diabetes mellitus (disorder), disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus (disorder), hyperlipidaemia due to type 2 diabetes mellitus, type ii diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus]","[5.7951, 5.7951, 6.1022, 6.2798, 6.3629, 6.6068, 6.7764, 6.8025, 6.8979, 6.9427]"


>>  unsp malignant neoplasm of lymph node
icd10cm_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,unsp malignant neoplasm of lymph node,0,36,None,C779,"[C779, C77, D360, C772, C775, D492, D219]","[secondary and unsp malignant neoplasm of lymph node, unsp, secondary and unspecified malignant neoplasm of lymph nodes, benign neoplasm of lymph node, secondary and unsp malignant neoplasm of intra-abd nodes, secondary and unsp malignant neoplasm of intrapelv nodes, neoplasm of lymph node, benign neoplasm of lymph vessels]","[5.9379, 6.2701, 6.7806, 7.2553, 7.6323, 7.6462, 7.8888]"


snomed_code


,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,unsp malignant neoplasm of lymph node,0,36,None,188379006,"[188379006, 271526003, 188432007, 92197001, 189203003, 188706002, 92202008, 92156008, 188707006, 188713002, 92090002, 92203003, 188392009, 188393004, 188709009, 188389005, 188715009, 92199003, 92213006, 127232002, 188423005, 92201001]","[secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes nos, benign neoplasm of lymph node, benign neoplasm of lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, benign neoplasm of lymph nodes of neck, benign neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, benign neoplasm of epitrochlear lymph nodes, benign neoplasm of lymph nodes of upper limb, secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck nos, secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, secondary and unspecified malignant neoplasm of submental lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, benign neoplasm of lymph nodes of head (disorder), benign neoplasm of mediastinal lymph nodes, neoplasm of lymph node, secondary and unspecified malignant neoplasm of intrapelvic lymph nodes, benign neoplasm of lymph nodes of multiple sites (disorder)]","[6.2701, 6.2701, 6.5341, 6.7806, 6.8029, 6.8955, 7.2334, 7.3035, 7.3339, 7.3721, 7.4309, 7.4429, 7.4456, 7.4954, 7.5084, 7.5154, 7.5664, 7.5726, 7.6336, 7.6462, 7.6835, 7.6909]"


##  How to integrate resolvers with NER models in the same pipeline


In [145]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

clinical_ner = NerDLModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['PROBLEM'])

c2doc = Chunk2Doc().setInputCols("ner_chunk").setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("icd10cm_code")\
  .setDistanceFunction("EUCLIDEAN")
  
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver])

data_ner = spark.createDataFrame([[text]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)



sentence_detector_dl download started this may take some time.
Approximate size to download 307.2 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [158]:

clinical_note = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

print (clinical_note)

clinical_note_df = spark.createDataFrame([[clinical_note]]).toDF("text")


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20

In [161]:

from pyspark.sql import functions as F

icd10_sdf = sbert_models.transform(clinical_note_df)

icd10_sdf = icd10_sdf.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata","icd10cm_code.result","icd10cm_code.metadata","ner_chunk.begin","ner_chunk.end")).alias("icd10cm_code")) \
    .select(F.expr("icd10cm_code['0']").alias("chunk"),
            F.expr("icd10cm_code['4']").alias("begin"),
            F.expr("icd10cm_code['5']").alias("end"),
            F.expr("icd10cm_code['1'].entity").alias("entity"),
            F.expr("icd10cm_code['2']").alias("code"),
            F.expr("icd10cm_code['3'].confidence").alias("confidence"),
            F.expr("icd10cm_code['3'].all_k_resolutions").alias("all_k_resolutions"),
            F.expr("icd10cm_code['3'].all_k_results").alias("all_k_codes"))

icd10_sdf.show()


+--------------------+-----+----+-------+-----+----------+--------------------+--------------------+
|               chunk|begin| end| entity| code|confidence|   all_k_resolutions|         all_k_codes|
+--------------------+-----+----+-------+-----+----------+--------------------+--------------------+
|gestational diabe...|   39|  67|PROBLEM|O2441|    0.9777|gestational diabe...|O2441:::O2443:::Z...|
|type two diabetes...|  128| 153|PROBLEM| E119|    0.6587|type 2 diabetes m...|E119:::E118:::O24...|
|              T2DM),|  156| 161|PROBLEM| E119|    0.1209|diabetes mellitus...|E119:::E109:::E88...|
|HTG-induced pancr...|  184| 207|PROBLEM|K8520|    0.2189|alcohol-induced p...|K8520:::K853:::K8...|
|  an acute hepatitis|  260| 277|PROBLEM| B159|    0.3662|acute hepatitis a...|B159:::K720:::B17...|
|             obesity|  284| 290|PROBLEM| E669|    0.9193|obesity:::abdomin...|E669:::E668:::Z68...|
|   a body mass index|  297| 313|PROBLEM|Z6841|    0.6263|finding of body m...|Z6841:::E669

In [ ]:
# with light pipeline

light_model = LightPipeline(sbert_models)

In [ ]:
!pip -q install spark-nlp-display

In [148]:
from sparknlp_display import EntityResolverVisualizer

vis = EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

light_data_icd = light_model.fullAnnotate(clinical_note)

vis.display(light_data_icd[0], 'ner_chunk', 'icd10cm_code')
